# Laboratorio 6. EAE253b
<hr/>

**Antes de comenzar:**

- Laboratorio debe ser realizado **de forma individual**. Obviamente, se pueden discutir ideas, pero cualquier intercambio de códigos **no está permitido**.
- Sólo se evaluarán secciones "**TAREAS**"; pero se recomienda realizar secciones **EJERCICIOS**

**Instrucciones de entrega:**

- Debe entregar este laboratorio por WebCurso, en buzón de tareas. Descargar archivo ".ipynb" a su equipo y luego subirlo.
- Además, **debe compartir el Notebook con ayudantes (arybertt@uc.cl, pagonzalez20@uc.cl) y profesor (cealvara@gmail.com)**.
- Plazo máximo de entrega: **Viernes 17 de mayo , 11.55 pm.**


<hr />

## Parte 1

En este laboratorio nuevamente trabajaremos con SQL. Esta vez con un enfoque algo más avanzado. 

Antes de partir, nos conectaremos a la base de datos (archivo de "sqlite3"), pero esta vez lo haremos a traves de Google Drive (Colab Research nos permite acceder a archivos que esten en Google Drive y leerlos desde Python).



In [0]:
# El siguiente codigo nos permite conectarnos a Google Drive. 

# Deben seguir las instrucciones que aparezcan en pantalla

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Una vez estamos conectados con google drive,  podemos cargar nuestra base de datos:

In [0]:
link = 'https://drive.google.com/file/d/1PkDtUMlkS8JtS0h3fbVa0n9pXM2u-k1i/view?usp=sharing'
id = link.split('/')[5]
print(id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('db.db')

In [0]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('db.db')
cursor = conn.cursor()

Ya estando conectados a la base de datos, podemos extraer informacion de ella como siempre:

In [0]:
#Inspeccionamos que tablas contiene la base de datos
a = 'SELECT tbl_name FROM sqlite_master where type="table"'
df = pd.read_sql(a,conn)
df

In [0]:
# Veamos qué columnas tiene la tabla "Visas"

ini = """
    select *
    from visas 
    limit 1
"""
df = pd.read_sql(ini,conn)
df

In [0]:
# Veamos qué columnas tiene la tabla "Estudios"

ini = """
    select *
    from Estudios
"""
df = pd.read_sql(ini,conn)
df

In [0]:
# Veamos qué columnas tiene la tabla "Sexo"

ini = """
    select *
    from Sexo
"""
df = pd.read_sql(ini,conn)
df

#### SQL basico: seleccion de columnas y filas

El uso más sencillo de SQL es acceder a ciertas columnas/variables, para ciertas filas/observaciones.  

En el siguiente ejemplo recuperamos el año de postulación, fecha de nacimiento, el país de origen y la región de origen de la base de datos de Visas. Además, sólo nos interesa acceder a información de personas con edad entre 15 y 30 años y que hayan postulado el año 2007 o antes. Finalmente, limitamos los resultados a las primeras 100 solicitudes que cumplan dicha condicion.

Nota: al igual que Python, Sqlite3 tiene una función llamada "strftime" que nos permite extraer elementos de una fecha. Para más opciones de dicha función, ver: https://www.techonthenet.com/sqlite/functions/strftime.php

In [0]:
ini = """
    select Id as "ID persona", 
        year as "Año Solicitud", 
        strftime('%Y',birth_date) as "Nacimiento",
        iso3 as "Cod. País",
        sub_region as "Sub Región"
    from visas 
    where age between 15 and 30 and year <=2007 
    limit 100
"""
df = pd.read_sql(ini,conn)
df

#### Consultas anidadas

Otra característica de SQL es que nos permite hacer consultas anidadas. Este tipo de consultas lo realizamos cuando queremos que SQL procese por 2da (o 3ra, o 4ta, ...) vez los resultados obtenidos.

En el siguiente ejemplo, primero calculamos la edad promedio de los postulantes y el número de postulantes, por país, y luego seleccionamos aquellos países donde la edad promedio está entre 26 y 30 años, y la cantidad de postulantes es mayor a 5.

In [0]:
#Ejemplo de consulta anidada
ini = """
select * 
from (
    select country_name as "País", 
        round(avg(age),0) as "Edad Promedio",
        count(1) as "Cantidad"
    from Visas 
    group by country_name
)
where ([Edad Promedio] between 26 and 30) and (Cantidad>5)
"""
df = pd.read_sql(ini,conn)
df

#### Creación de variables "avanzada": Expresión "CASE"

Ya hemos visto ejemplos para crear variables en SQL. Por ejemplo, si quisieramos crear una tabla con la variable "año de nacimiento", lo hariamos de la siguiente forma:

In [0]:
ini = """
    select strftime('%Y',birth_date) as "Nacimiento"
    from visas 
    limit 100
"""
df = pd.read_sql(ini,conn)
df

Sin embargo, existen ocasiones en que la variable que queremos crear toma distintos valores según los valores de otras variables. El caso más simple, sería la creación de una variable "dummy", donde queremos que dicha columna sea 1 en ciertos casos, y sea 0 en otros. 

En el siguiente ejemplo, creamos una columna llamanda "dummy_peru", que marca las observaciones donde el país de origen es Perú.

Aqui se puede aprender más de la expresión "CASE"

http://www.sqlitetutorial.net/sqlite-case/

In [0]:
query = '''
    select case
        when iso3 = "PER" then 1 
        else 0 
        end as dummy_peru

    from visas
    
'''
df = pd.read_sql(query, conn)
df

#### Eliminando filas: sentencia "EXCEPT"

En ciertas ocasiones, queremos "filtrar" los resultados obtenidos para que no contengan los resultados de otra consulta determinada. En estos casos, utilizamos la sentencia "EXCEPT". Esta sentencia deja todas las observaciones de la primera consulta que no se encuentren en la segunda consulta.

*Nota: este comportamiento se podría lograr con el uso de WHERE y condiciones más complejas; sin embargo, la sentencia EXCEPT también nos permite filtrar resultados que provengan de tablas distintas (en cuyo caso, no podríamos usar "WHERE")*

In [0]:
#Ejemplo de except
ini = """
select * from Visas where (visa_type_id in (1,2)) and (age>70)
except
select * from Visas where (studies_id between 0 and 5) or (activity_id<8)
"""
df = pd.read_sql(ini,conn)
df

### Tarea 1.

<hr>


1. Escriba una consulta que le permita calcular, por país de origen, el promedio de edad **actual** y el número (*count*) de los postulantes con educación técnica o universitaria". (*Hint: notar que la variable "age" es una variable "estática", ya que se refiere a la edad al momento de ingresar el registro a la base de datos. Esto no nos sirve si queremos poder comparar la edad de personas que se registraron en distintos momentos del tiempo)*. No asuma que la edad actual es "2019 - año nacimiento".

2. ¿cómo se comparan dichas estadísticas (promedio y número) con aquellas de postulantes con un nivel de educación menor a técnica o universitaria? Para responder esto, debería crear una tabla con 5 columnas:

  - la primera columna debe ser el codigo de país, 
  - la segunda y tercera columna deben ser el promedio de edad y el número de postulantes con educación técnica o universitaria
  - la cuarta y quinta columna deben ser el promedio de edad y el número de postulantes con educación menor a técnica o universitaria.

*(Para responder esta parte, basta con crear la tabla mencionada)*

3. Escriba código que le permita saber los 3 países con mayor % de postulantes con bajo nivel de educación (respecto a los postulantes totales de ese país).


4. Escriba código que le permita saber los 3 países donde diferencia de edad de postulantes con bajo nivel de educación y alto nivel de educación sea la mayor. Vale decir, queremos saber desde qué países provienen los postulantes con alto nivel de educación **más jovenes**, respecto de sus pares con bajo nivel de educación.
 

5. Ahora, para cada grupo de género, queremos saber la cantidad de países desde donde provienen, al igual que la cantidad de edades distintas que tienen.

6. Como puede ver de la tabla anterior, existen países donde los postulantes son exclusivamente hombres. También podría haber países donde las postulantes sean exclusivamente mujeres. Escriba código que le permita determinar cuáles son esos países.

## Parte 2.

<hr>

En esta parte, analizaremos datos del Servicio de Impuestos Internos (SII). La base de datos a la que accederemos contiene un extracto de la "Nómina de empresas personas jurídicas AT 2006-2017" disponible en http://www.sii.cl/sobre_el_sii/nominapersonasjuridicas.html. Basicamente, es un registro de empresas con cierta informacion de las empresas. La información está organizada en tablas para cada año.

In [0]:
link = 'https://drive.google.com/file/d/1iEbeNcdHYJ2D-JWi0cT5H2ccDefzDroF/view?usp=sharing'
id = link.split('/')[5]
print(id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('sii.db')



**Cargamos base de datos y vemos que tablas contiene.**

In [0]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('sii.db')
cursor = conn.cursor()

In [0]:
#Inspeccionamos que tablas contiene la base de datos
a = 'SELECT tbl_name FROM sqlite_master where type="table"'
df = pd.read_sql(a,conn)
df

***Imprimamos algunos registros para ver qué variables contiene una tabla. En este caso veremos para SII_2013.*** <br/>
Recuerde que quiera aprender más y practicar, puede utilizar sitios web como el [siguiente](https://www.w3schools.com/sql/)

In [0]:
a = '''
    
'''

df = pd.read_sql(a,conn)
df

### Tarea 2.

<hr>


1. ¿Existen Ruts duplicados en la tabla SII_2013? ¿a qué cree usted que se debe?

In [0]:
#Inserte su código aquí

2. Cree una función (Python) que le permita saber el número de observaciones duplicadas de alguna tabla particular, para una lista de columnas particular de esa tabla. Por ejemplo, la pregunta anterior se respondería así:


```
num_duplicados_2013 = encuentra_duplicados(conn, 'SII_2013', ['Cliente_Rut'])
```

donde *conn* es la conexión a la base de datos.



In [0]:
#Inserte su código aquí

3. Busque el número de observaciones duplicadas de las tablas SII_*.



In [0]:
#Inserte su código aquí

4. Cree una nueva función, similar a "encuentra_duplicados", pero que en vez de retornar el número de duplicados, le entregue un DataFrame con las observaciones que están duplicadas. Vale decir, su función se usaría asi:


```
df_duplicados_2013 = encuentra_duplicados(conn, 'SII_2013', ['Cliente_Rut'])
```

In [0]:
#Inserte su código aquí

5. Encuentre la lista de observaciones duplicadas de la tabla SII_2015, para los campos Cliente_Rut y Tipo_Contribuyente.

In [0]:
#Inserte su código aquí

6. Encuentre la lista de observaciones duplicadas de la tabla SII_2016, para los campos Cliente_Rut y Fecha_Termino_Giro.

In [0]:
#Inserte su código aquí

7. ¿Puede decir ahora a qué se deben las observaciones duplicadas?

Respuesta:

<hr/>
<center> <h1>Fin del laboratorio.</h1> </center>

 **Recuerde compartir el Notebook con ayudantes (arybertt@uc.cl, pagonzalez20@uc.cl) y profesor (cealvara@gmail.com)**.